In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [38]:
import glob
path = "./../dist/char*.txt"
contents=[]
for filename in glob.glob(path):
    with open(filename, 'r') as f:
        for line in f:
            contents.append(line.strip('\n'))
print('Liczba plików: ', len(contents)/15)
i=0
while (i<len(contents)-1):
    contents[i]=float(contents[i])
    i+=1
    if ((i+1)%15==0):
        i=i+1

Liczba plików:  125.0


In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [40]:
from random import randint as r

In [41]:
# while True:
#     a=r(0, (len(contents)/15))
#     print(a)
#     print(torch.tensor(contents[(a*15):(a*15+14)]))

In [42]:
import time

In [43]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(in_features=14, out_features=14)
        self.fc2 = nn.Linear(in_features=14, out_features=4)
        #self.out = nn.Linear(in_features=14, out_features = 4, bias=True)
        
    def forward(self, t):
        #1st layer
        t = self.fc1(t)
        #2nd layer
        t = self.fc2(t)

        #t = F.softmax(t)
        
        return t

In [44]:
net=Network()
params=list(net.parameters())
criterion = nn.MSELoss()
net.zero_grad()
optimizer = optim.SGD(net.parameters(), lr=0.000000001)
lossDuplicationCounter = 0
prevloss=5
loss = 4
i=0
good = 0

In [45]:
start=time.time()
while (good < 10):
    if float(prevloss)<1:
        good+=1
    else:
        good = 0
        
    a=r(0, (len(contents)/15-1))
    chars = torch.tensor(contents[(a*15):(a*15+14)])
    optimizer.zero_grad()   # zero the gradient buffers
    out = net(chars)
    
    start=a*15
    #print(contents[start+14])
    if (contents[start+14]=='Marcin'):
        target = torch.tensor([1., 0., 0., 0.])
    elif (contents[start+14]=='Bartek'):
        target = torch.tensor([0., 1., 0., 0.])
    elif (contents[start+14]=='Maciek'):
        target = torch.tensor([0., 0., 1., 0.,])
    elif (contents[start+14]=='Artur'):
        target = torch.tensor([0., 0., 0., 1.,])
    loss = criterion(out, target)
    #print('loss', float(loss))
    if prevloss == loss:
        lossDuplicationCounter +=1
        print('loss duplicate #', lossDuplicationCounter)
    else:
        lossDuplicationCounter = 0
    prevloss = loss
    loss.backward(retain_graph=True)
    #loss.backward()
    optimizer.step()   # Does the update
    if i%1000==0:
        print("\n\niteration: ", i, 
              "\nLoss: ", ((str(loss)).strip('tensor()')).strip(', grad_fn=<MseLossBackward>) '), 
              '\nOutput: ', ((str(out)).strip('tensor(')).strip(', grad_fn=<AddBackward0>)'),
              '\nTarget: ', str(target).strip('tensor(').strip(')'))
    i+=1
print("Learning time: \n\t", time.time()-start)
torch.cuda.empty_cache()



iteration:  0 
Loss:  1.1125e+08 
Output:  [ -3610.8003, -15048.6631, -14250.2402,   1565.6216],
 
Target:  [0., 0., 0., 1.]


iteration:  1000 
Loss:  168995.4688 
Output:  [-193.9147, -359.6286, -674.1716, -230.6286] 
Target:  [0., 0., 1., 0.]


iteration:  2000 
Loss:  7035.9116 
Output:  [-13.2299, 122.7658,  72.7841,  88.1760] 
Target:  [0., 0., 0., 1.]


iteration:  3000 
Loss:  41.1001 
Output:  [ 0.5807, 10.0269, -2.0258,  8.8586] 
Target:  [0., 1., 0., 0.]


iteration:  4000 
Loss:  2240.0986 
Output:  [-17.7138, -39.1847, -84.1103,  -0.4564] 
Target:  [1., 0., 0., 0.]


iteration:  5000 
Loss:  937.6060 
Output:  [-18.1604,  26.4323,  14.0634, -49.2412] 
Target:  [0., 0., 0., 1.]


iteration:  6000 
Loss:  36.1628 
Output:  [ 9.7607, -5.7863,  3.9377,  0.3722] 
Target:  [0., 0., 0., 1.]


iteration:  7000 
Loss:  37.0070 
Output:  [ 1.6326,  2.3806,  3.7165, 12.2198] 
Target:  [0., 0., 0., 1.]


iteration:  8000 
Loss:  44.4864 
Output:  [-4.7857,  7.0916, -2.1375, -9.7421]



iteration:  75000 
Loss:  1.0068 
Output:  [-0.0763, -0.7447,  1.5186, -0.0891] 
Target:  [1., 0., 0., 0.]


iteration:  76000 
Loss:  23.5819 
Output:  [-4.8161,  8.5801, -2.1735, -2.9919] 
Target:  [0., 1., 0., 0.]


iteration:  77000 
Loss:  0.1578 
Output:  [-0.1539, -0.6304,  0.4470,  0.8979] 
Target:  [0., 0., 0., 1.]


iteration:  78000 
Loss:  10.6676 
Output:  [ 2.7381, -3.1157, -0.5427,  4.2355] 
Target:  [0., 1., 0., 0.]


iteration:  79000 
Loss:  26.0278 
Output:  [ 6.5276, -7.4922,  2.3819,  3.4279] 
Target:  [1., 0., 0., 0.]


iteration:  80000 
Loss:  2.2599 
Output:  [ 1.0924, -1.3752, -0.2129,  2.1175] 
Target:  [0., 0., 1., 0.]


iteration:  81000 
Loss:  1.8312 
Output:  [-1.6250,  1.4725, -0.5448,  0.3600] 
Target:  [0., 0., 1., 0.]


iteration:  82000 
Loss:  4.6755 
Output:  [ 2.7427, -1.8763, -0.0731,  1.7032] 
Target:  [0., 1., 0., 0.]


iteration:  83000 
Loss:  1.5679 
Output:  [ 1.5716, -1.5903,  0.5108,  1.0166] 
Target:  [0., 0., 1., 0.]


iteration:  84

KeyboardInterrupt: 

In [ ]:
print(prevloss)

In [ ]:
print("\n\niteration: ", i, 
              "\nLoss: ", ((str(loss)).strip('tensor()')).strip(', grad_fn=<MseLossBackward>) '), 
              '\nOutput: ', ((str(out)).strip('tensor(')).strip(', grad_fn=<AddBackward0>)').strip(', grad_fn=<Softmax'),
              '\nTarget: ', str(target).strip('tensor(').strip(')'))

In [ ]:
print
i=0
for filename in glob.glob(path):
    #a=r(0, (len(contents)/15-1))
    chars = torch.tensor(contents[(i*15):(i*15+14)])
    #print(str(net(chars)).strip(", grad_fn=<AddBackward0>)").strip("tensor("), "\t : ",contents[i*15+14])
    print(((str(net(chars))).strip('tensor(')).strip(', grad_fn=<AddBackward0>)').strip(', grad_fn=<Softmax'), " : ", contents[i*15+14])
    i=i+1

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
torch.save(net, "net.pt")


In [ ]:
haba = torch.load("net.pt")
haba.eval()


In [ ]:


i=0
ilezg=0
for filename in glob.glob(path):
    tt=torch.tensor(contents[(a*15):(a*15+14)])
    who = contents[i*15+14]
    predykcja = haba(tt)
    print(str(predykcja).strip('tensor(').strip(', grad_fn=<AddBackward0>)').strip(', grad_fn=<SoftmaxBackward>)')) 
    predlist = predykcja.tolist().index(max(predykcja.tolist()))
    predykcjaOsoba=''
    if predlist == 0:
        predykcjaOsoba='Marcin'
    elif predlist == 1:
        predykcjaOsoba='Bartek'
    elif predlist == 2:
        predykcjaOsoba='Maciek'
    elif predlist == 3:
        predykcjaOsoba='Artur'
    else:
        print('ERROR!!!')
    print(i, who, ' : \n', str(tt).strip('tensor(').strip(')').strip(', grad_fn=<Softmax')), #'\n', str(predykcja).strip('tensor(').strip(', grad_fn=<AddBackward0>)').strip(', grad_fn=<SoftmaxBackward>)'))
    i+=1
    if who == predykcjaOsoba:
        ilezg+=1
print(ilezg/i*100)
